In [2]:
import pydantic
import pandas as pd

## Primer juego

Es temporada de oscares;  Pedro (P) y Marisela (M) desean apostar sobre cual va a ser la ganadora de mejor pelicula. Pedro dice que la sustancia va será la ganadora mientras que Marisela apuesta a  Conclave. Plantean lo siguiente: si ninguna de esas dos peliculas gana entonces ambos pierden su dinero más un 2% del dinero que estan apostando, esta acción les brinda -8 de utilidad a ambos. Si gana la sustancia etnocnes Pedro recibe 5 de utilidad y Marisela 2; si Conclave gana pasa lo contrario. ¿Cuál es la mejor estrategia que se debe de lograr?, ¿Hay estrategias dominantes? 

In [2]:
from typing import Dict, List, Tuple, Union, Optional
from pydantic import BaseModel, Field, validator

# 1. Estructura de datos en pydantic
class Estrategias(BaseModel):
    """Clase que representa las estrategias disponibles para cada jugador"""
    nombre: str
    opciones: List[str]

class Pagos(BaseModel):
    """Clase que representa los pagos para una combinación de estrategias"""
    valores: Dict[str, float]
    
    @validator('valores')
    def validar_jugadores(cls, v):
        if not v:
            raise ValueError("Debe haber al menos un jugador con pago")
        return v

class EstructuraDeJuego(BaseModel):
    """Clase principal que define un juego en forma normal"""
    nombre: str
    jugadores: List[str]
    estrategias: Dict[str, Estrategias]
    matriz_pagos: List[Dict[Tuple[str, ...], Pagos]]
    
    @validator('matriz_pagos')
    def validar_matriz_pagos(cls, v, values):
        if 'jugadores' not in values:
            return v
        
        # Verificar que todas las combinaciones de estrategias estén definidas
        jugadores = values['jugadores']
        estrategias = values.get('estrategias', {})
        
        if not estrategias:
            return v
            
        # Verificar que cada jugador tenga pagos definidos
        for combinacion_pagos in v:
            for combinacion, pagos in combinacion_pagos.items():
                if len(combinacion) != len(jugadores):
                    raise ValueError(f"La combinación {combinacion} no tiene el número correcto de estrategias")
                for jugador in jugadores:
                    if jugador not in pagos.valores:
                        raise ValueError(f"Falta el pago para {jugador} en la combinación {combinacion}")
        
        return v

# 2. Función para calcular pagos
def pagos_estrategia(juego: EstructuraDeJuego, estrategias: Dict[str, str]) -> Pagos:
    """
    Calcula los pagos para una combinación de estrategias
    
    Args:
        juego: La estructura del juego
        estrategias: Diccionario con la estrategia elegida por cada jugador
        
    Returns:
        Los pagos correspondientes a la combinación de estrategias
    """
    # Verificar que todos los jugadores tengan una estrategia asignada
    for jugador in juego.jugadores:
        if jugador not in estrategias:
            raise ValueError(f"Falta la estrategia para el jugador {jugador}")
        
        # Verificar que la estrategia sea válida
        if estrategias[jugador] not in juego.estrategias[jugador].opciones:
            raise ValueError(f"Estrategia inválida {estrategias[jugador]} para el jugador {jugador}")
    
    # Encontrar los pagos correspondientes a la combinación de estrategias
    combinacion = tuple(estrategias[jugador] for jugador in juego.jugadores)
    
    for matriz in juego.matriz_pagos:
        if combinacion in matriz:
            return matriz[combinacion]
    
    raise ValueError(f"No se encontraron pagos para la combinación {combinacion}")

# 3. Algoritmo para encontrar equilibrios de Nash
def encontrar_equilibrios_nash(juego: EstructuraDeJuego) -> List[Tuple[Dict[str, str], Pagos]]:
    """
    Encuentra todos los equilibrios de Nash de estrategia pura en un juego
    
    Args:
        juego: La estructura del juego
        
    Returns:
        Lista de tuplas (estrategias, pagos) correspondientes a los equilibrios de Nash
    """
    equilibrios = []
    
    # Generar todas las posibles combinaciones de estrategias
    def generar_combinaciones(jugadores, idx=0, estrategias_actuales=None):
        if estrategias_actuales is None:
            estrategias_actuales = {}
            
        if idx == len(jugadores):
            # Verificar si esta combinación es un equilibrio de Nash
            if es_equilibrio_nash(juego, estrategias_actuales):
                pagos = pagos_estrategia(juego, estrategias_actuales)
                equilibrios.append((estrategias_actuales.copy(), pagos))
            return
        
        jugador = jugadores[idx]
        for estrategia in juego.estrategias[jugador].opciones:
            estrategias_actuales[jugador] = estrategia
            generar_combinaciones(jugadores, idx + 1, estrategias_actuales)
    
    generar_combinaciones(juego.jugadores)
    return equilibrios

def es_equilibrio_nash(juego: EstructuraDeJuego, estrategias: Dict[str, str]) -> bool:
    """
    Verifica si una combinación de estrategias es un equilibrio de Nash
    
    Args:
        juego: La estructura del juego
        estrategias: Diccionario con la estrategia elegida por cada jugador
        
    Returns:
        True si es un equilibrio de Nash, False en caso contrario
    """
    pagos_originales = pagos_estrategia(juego, estrategias)
    
    # Para cada jugador, verificar si puede mejorar cambiando su estrategia
    for jugador in juego.jugadores:
        pago_original = pagos_originales.valores[jugador]
        
        # Probar todas las estrategias alternativas del jugador
        estrategia_original = estrategias[jugador]
        for estrategia_alternativa in juego.estrategias[jugador].opciones:
            if estrategia_alternativa == estrategia_original:
                continue
                
            # Cambiar la estrategia del jugador
            estrategias_alternativas = estrategias.copy()
            estrategias_alternativas[jugador] = estrategia_alternativa
            
            try:
                pagos_alternativos = pagos_estrategia(juego, estrategias_alternativas)
                pago_alternativo = pagos_alternativos.valores[jugador]
                
                # Si el jugador puede mejorar, no es un equilibrio de Nash
                if pago_alternativo > pago_original:
                    return False
            except ValueError:
                # Si la combinación no existe, continuamos
                continue
    
    return True

# Definición del juego de Pedro y Marisela
def crear_juego_oscares():
    """Crea el juego de apuestas sobre los Óscares entre Pedro y Marisela"""
    return EstructuraDeJuego(
        nombre="Apuesta de Óscares",
        jugadores=["Pedro", "Marisela"],
        estrategias={
            "Pedro": Estrategias(
                nombre="Apuesta de Pedro",
                opciones=["La Sustancia", "Conclave", "Otra"]
            ),
            "Marisela": Estrategias(
                nombre="Apuesta de Marisela",
                opciones=["La Sustancia", "Conclave", "Otra"]
            )
        },
        matriz_pagos=[{
            # (Pedro, Marisela)
            ("La Sustancia", "La Sustancia"): Pagos(valores={"Pedro": 5, "Marisela": 2}),
            ("La Sustancia", "Conclave"): Pagos(valores={"Pedro": 5, "Marisela": 2}),
            ("La Sustancia", "Otra"): Pagos(valores={"Pedro": 5, "Marisela": 2}),
            
            ("Conclave", "La Sustancia"): Pagos(valores={"Pedro": 2, "Marisela": 5}),
            ("Conclave", "Conclave"): Pagos(valores={"Pedro": 2, "Marisela": 5}),
            ("Conclave", "Otra"): Pagos(valores={"Pedro": 2, "Marisela": 5}),
            
            ("Otra", "La Sustancia"): Pagos(valores={"Pedro": -8, "Marisela": -8}),
            ("Otra", "Conclave"): Pagos(valores={"Pedro": -8, "Marisela": -8}),
            ("Otra", "Otra"): Pagos(valores={"Pedro": -8, "Marisela": -8}),
        }]
    )

# Análisis del juego
def analizar_juego_oscares():
    """Analiza el juego de apuestas sobre los Óscares"""
    juego = crear_juego_oscares()
    print(f"Juego: {juego.nombre}")
    print(f"Jugadores: {juego.jugadores}")
    
    print("\nEstrategias:")
    for jugador, estrategias in juego.estrategias.items():
        print(f"  {jugador}: {estrategias.opciones}")
    
    print("\nMatriz de pagos:")
    for jugador in juego.jugadores:
        print(f"  {jugador}:")
        for combinacion, pagos in juego.matriz_pagos[0].items():
            print(f"    {combinacion}: {pagos.valores[jugador]}")
    
    print("\nEquilibrios de Nash:")
    equilibrios = encontrar_equilibrios_nash(juego)
    if not equilibrios:
        print("  No se encontraron equilibrios de Nash de estrategia pura.")
    else:
        for i, (estrategias, pagos) in enumerate(equilibrios, 1):
            print(f"  Equilibrio {i}:")
            for jugador, estrategia in estrategias.items():
                print(f"    {jugador}: {estrategia}")
            print(f"    Pagos: {pagos.valores}")
    
    # Verificar estrategias dominantes
    print("\nAnálisis de estrategias dominantes:")
    for jugador in juego.jugadores:
        print(f"  Para {jugador}:")
        tiene_dominante = False
        
        for estrategia in juego.estrategias[jugador].opciones:
            es_dominante = True
            
            # Verificar si esta estrategia domina a todas las demás
            for otra_estrategia in juego.estrategias[jugador].opciones:
                if otra_estrategia == estrategia:
                    continue
                
                # Verificar para todas las combinaciones de estrategias del otro jugador
                for i, otro_jugador in enumerate(juego.jugadores):
                    if otro_jugador == jugador:
                        continue
                    
                    for estrategia_otro in juego.estrategias[otro_jugador].opciones:
                        # Estrategia a evaluar
                        estrategias_eval = {jugador: estrategia, otro_jugador: estrategia_otro}
                        pagos_eval = pagos_estrategia(juego, estrategias_eval)
                        
                        # Estrategia alternativa
                        estrategias_alt = {jugador: otra_estrategia, otro_jugador: estrategia_otro}
                        pagos_alt = pagos_estrategia(juego, estrategias_alt)
                        
                        # Si la estrategia alternativa da mejor pago, no es dominante
                        if pagos_alt.valores[jugador] > pagos_eval.valores[jugador]:
                            es_dominante = False
                            break
                    
                    if not es_dominante:
                        break
                
                if not es_dominante:
                    break
            
            if es_dominante:
                tiene_dominante = True
                print(f"    '{estrategia}' es una estrategia dominante para {jugador}")
        
        if not tiene_dominante:
            print(f"    No tiene estrategia dominante")

# Ejecutar el análisis
if __name__ == "__main__":
    analizar_juego_oscares()

Juego: Apuesta de Óscares
Jugadores: ['Pedro', 'Marisela']

Estrategias:
  Pedro: ['La Sustancia', 'Conclave', 'Otra']
  Marisela: ['La Sustancia', 'Conclave', 'Otra']

Matriz de pagos:
  Pedro:
    ('La Sustancia', 'La Sustancia'): 5.0
    ('La Sustancia', 'Conclave'): 5.0
    ('La Sustancia', 'Otra'): 5.0
    ('Conclave', 'La Sustancia'): 2.0
    ('Conclave', 'Conclave'): 2.0
    ('Conclave', 'Otra'): 2.0
    ('Otra', 'La Sustancia'): -8.0
    ('Otra', 'Conclave'): -8.0
    ('Otra', 'Otra'): -8.0
  Marisela:
    ('La Sustancia', 'La Sustancia'): 2.0
    ('La Sustancia', 'Conclave'): 2.0
    ('La Sustancia', 'Otra'): 2.0
    ('Conclave', 'La Sustancia'): 5.0
    ('Conclave', 'Conclave'): 5.0
    ('Conclave', 'Otra'): 5.0
    ('Otra', 'La Sustancia'): -8.0
    ('Otra', 'Conclave'): -8.0
    ('Otra', 'Otra'): -8.0

Equilibrios de Nash:
  Equilibrio 1:
    Pedro: La Sustancia
    Marisela: La Sustancia
    Pagos: {'Pedro': 5.0, 'Marisela': 2.0}
  Equilibrio 2:
    Pedro: La Sustancia
   

/var/folders/1_/0gj_v5x54fxd8gllt64jsbp80000gn/T/ipykernel_13576/4134225701.py:14: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('valores')
/var/folders/1_/0gj_v5x54fxd8gllt64jsbp80000gn/T/ipykernel_13576/4134225701.py:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('matriz_pagos')


# Segundo juego

Mariela y Fabiola son dos cientificas que trabajan en el acelerador de particulas. Estan prontas a sacar los resultados delos experimentos pero llevan tanto tiempo concentradas que no han podido comer. La carga de trabajo es tanta que el tiempo y la presión que el tiempo que se tome para comer es tiempo que no se utiliza para el proyecto. Deciden que pueden realizar las diversas acciones: si las dos comen entonces cada una recibe 5 unidades de utilidad. Si solo una come y la otra no entonces la que come recibe 7 puntos y la otra no recibe utilidad. ¿Cuál es la mejor estrategia?

In [6]:
# Asegúrate de tener instalada la versión 2 de pydantic:
# pip install "pydantic>=2.0"

from pydantic import BaseModel, Field, model_validator
from typing import List, Dict, Tuple, Any

# Definición de la clase Estrategia
class Estrategia(BaseModel):
    nombre: str

# Definición de la estructura principal del juego
class EstructuraDeJuego(BaseModel):
    nombre: str = Field(..., description="Nombre del juego")
    descripcion: str = Field(..., description="Descripción narrativa del juego")
    jugadores: List[str] = Field(..., description="Lista de jugadores")
    estrategias: Dict[str, List[Estrategia]] = Field(..., description="Estrategias disponibles por jugador")
    # Pagos definidos para cada perfil (clave: tupla de estrategias) y expresados como una tupla con el pago para cada jugador.
    pagos: Dict[Tuple[str, ...], Tuple[float, ...]] = Field(..., description="Pagos para cada combinación de estrategias")

    @model_validator(mode="after")
    def check_game_validity(self):
        jugadores = self.jugadores
        estrategias = self.estrategias
        pagos = self.pagos
        # Cada jugador debe tener al menos una estrategia
        if not all(j in estrategias and len(estrategias[j]) > 0 for j in jugadores):
            raise ValueError("Cada jugador debe tener al menos una estrategia.")
        # Verifica el número esperado de perfiles (producto cartesiano de las estrategias)
        expected_profiles = 1
        for j in jugadores:
            expected_profiles *= len(estrategias[j])
        if len(pagos) != expected_profiles:
            raise ValueError(f"El número de perfiles de estrategias ({len(pagos)}) no coincide con el esperado ({expected_profiles}).")
        # Verifica que cada clave y su correspondiente pago tengan la longitud correcta
        for key, valor in pagos.items():
            if len(key) != len(jugadores):
                raise ValueError(f"La clave {key} no coincide con el número de jugadores ({len(jugadores)}).")
            if len(valor) != len(jugadores):
                raise ValueError(f"Los pagos {valor} no coinciden con el número de jugadores ({len(jugadores)}).")
        return self

# Función que, dado un perfil de estrategias, retorna los pagos correspondientes
def pagos_estrategia(juego: EstructuraDeJuego, perfil: Tuple[str, ...]) -> Tuple[float, ...]:
    if perfil not in juego.pagos:
        raise ValueError(f"El perfil {perfil} no está definido en los pagos del juego.")
    return juego.pagos[perfil]

# Función para extraer las matrices de pago (asumiendo un juego de dos jugadores)
def get_payoff_matrices(juego: EstructuraDeJuego):
    jugador1, jugador2 = juego.jugadores[0], juego.jugadores[1]
    estrategiasA = [e.nombre for e in juego.estrategias[jugador1]]
    estrategiasB = [e.nombre for e in juego.estrategias[jugador2]]
    m = len(estrategiasA)
    n = len(estrategiasB)
    # Inicializamos las matrices
    payoffA = [[0 for _ in range(n)] for _ in range(m)]
    payoffB = [[0 for _ in range(n)] for _ in range(m)]
    for i, eA in enumerate(estrategiasA):
        for j, eB in enumerate(estrategiasB):
            key = (eA, eB)
            if key not in juego.pagos:
                raise ValueError(f"No hay pago definido para el perfil {key}.")
            p1, p2 = juego.pagos[key]
            payoffA[i][j] = p1
            payoffB[i][j] = p2
    return payoffA, payoffB, estrategiasA, estrategiasB

# Función para hallar equilibrios de Nash en estrategias puras según el pseudocódigo proporcionado
def findPureStrategyNashEquilibria(payoffA: List[List[float]], payoffB: List[List[float]]) -> List[Tuple[int, int]]:
    equilibria = []
    m = len(payoffA)
    n = len(payoffA[0]) if m > 0 else 0
    for i in range(m):
        for j in range(n):
            # Checamos si Mariela (jugador 1) puede mejorar cambiando su estrategia
            canAImprove = False
            for i_prime in range(m):
                if payoffA[i_prime][j] > payoffA[i][j]:
                    canAImprove = True
                    break
            # Checamos si Fabiola (jugador 2) puede mejorar cambiando su estrategia
            canBImprove = False
            for j_prime in range(n):
                if payoffB[i][j_prime] > payoffB[i][j]:
                    canBImprove = True
                    break
            if (not canAImprove) and (not canBImprove):
                equilibria.append((i, j))
    return equilibria

# Función que envuelve la búsqueda de equilibrios y retorna, para cada equilibrio, el perfil de estrategias y sus pagos
def nash_equilibria(juego: EstructuraDeJuego) -> List[Dict[str, Any]]:
    payoffA, payoffB, estrategiasA, estrategiasB = get_payoff_matrices(juego)
    indices_eq = findPureStrategyNashEquilibria(payoffA, payoffB)
    resultados = []
    for (i, j) in indices_eq:
        perfil = (estrategiasA[i], estrategiasB[j])
        pago = (payoffA[i][j], payoffB[i][j])
        resultados.append({
            "perfil": perfil,
            "pagos": pago
        })
    return resultados

# -----------------------
# Definición del juego: Mariela y Fabiola en el acelerador de partículas
jugadores = ["Mariela", "Fabiola"]
estrategias = {
    "Mariela": [Estrategia(nombre="Comer"), Estrategia(nombre="No comer")],
    "Fabiola": [Estrategia(nombre="Comer"), Estrategia(nombre="No comer")]
}
# Pagos:
# - ("Comer", "Comer") -> (5, 5)
# - ("Comer", "No comer") -> (7, 0)
# - ("No comer", "Comer") -> (0, 7)
# - ("No comer", "No comer") -> (0, 0)
pagos = {
    ("Comer", "Comer"): (5, 5),
    ("Comer", "No comer"): (7, 0),
    ("No comer", "Comer"): (0, 7),
    ("No comer", "No comer"): (0, 0)
}

juego_cientificas = EstructuraDeJuego(
    nombre="Mariela y Fabiola en el acelerador",
    descripcion=(
        "Dos científicas que trabajan en un acelerador de partículas deben decidir si toman un descanso para comer o no. "
        "Si ambas comen, cada una recibe 5 unidades de utilidad. Si solo una come, la que come recibe 7 y la otra 0. "
        "Si ninguna come, ninguna obtiene utilidad."
    ),
    jugadores=jugadores,
    estrategias=estrategias,
    pagos=pagos
)

print("Juego definido correctamente:")
print(juego_cientificas)

# Buscar y mostrar los equilibrios de Nash
equilibrios = nash_equilibria(juego_cientificas)
print("\nEquilibrios de Nash encontrados:")
for eq in equilibrios:
    print(eq)


Juego definido correctamente:
nombre='Mariela y Fabiola en el acelerador' descripcion='Dos científicas que trabajan en un acelerador de partículas deben decidir si toman un descanso para comer o no. Si ambas comen, cada una recibe 5 unidades de utilidad. Si solo una come, la que come recibe 7 y la otra 0. Si ninguna come, ninguna obtiene utilidad.' jugadores=['Mariela', 'Fabiola'] estrategias={'Mariela': [Estrategia(nombre='Comer'), Estrategia(nombre='No comer')], 'Fabiola': [Estrategia(nombre='Comer'), Estrategia(nombre='No comer')]} pagos={('Comer', 'Comer'): (5.0, 5.0), ('Comer', 'No comer'): (7.0, 0.0), ('No comer', 'Comer'): (0.0, 7.0), ('No comer', 'No comer'): (0.0, 0.0)}

Equilibrios de Nash encontrados:
{'perfil': ('Comer', 'Comer'), 'pagos': (5.0, 5.0)}


# Tercer juego

Aitana es una escultora que realizo una exposición sobre la neocolonizacion. Ella tiene la opción de poder presentarse en el museo britanico, pero puede ser sumamente criticada por eso: puede ser critica positica o negativo. Si decide exponer en el museo birtanico y que tenga una critica positiva es de 7,5 respectivamente. El no exponer en el museo britanico y que sea criticada positicamente ofrece una utilidad 5,4. Si expone en el museo britanico y que sea criticada negativa ofrece utilidad de 9,8; pero que sea criticada negativamente y que no exponga en el museo britanico es de 8,2. 

In [7]:
# Asegúrate de tener instalada la versión 2 de pydantic:
# pip install "pydantic>=2.0"

from pydantic import BaseModel, Field, model_validator
from typing import List, Dict, Tuple, Any

# ---------------------------
# Definición de modelos

class Estrategia(BaseModel):
    nombre: str

class EstructuraDeJuego(BaseModel):
    nombre: str = Field(..., description="Nombre del juego")
    descripcion: str = Field(..., description="Descripción narrativa del juego")
    jugadores: List[str] = Field(..., description="Lista de jugadores")
    estrategias: Dict[str, List[Estrategia]] = Field(..., description="Estrategias disponibles por jugador")
    # Los pagos se definen para cada perfil de estrategias (clave: tupla de estrategias) y se expresan
    # como una tupla con el pago para cada jugador, siguiendo el orden en la lista de jugadores.
    pagos: Dict[Tuple[str, ...], Tuple[float, ...]] = Field(..., description="Pagos para cada combinación de estrategias")

    @model_validator(mode="after")
    def check_game_validity(self):
        jugadores = self.jugadores
        estrategias = self.estrategias
        pagos = self.pagos
        # Cada jugador debe tener al menos una estrategia
        if not all(j in estrategias and len(estrategias[j]) > 0 for j in jugadores):
            raise ValueError("Cada jugador debe tener al menos una estrategia.")
        # Verifica que el número de perfiles coincida con el producto cartesiano de las estrategias
        expected_profiles = 1
        for j in jugadores:
            expected_profiles *= len(estrategias[j])
        if len(pagos) != expected_profiles:
            raise ValueError(f"El número de perfiles de estrategias ({len(pagos)}) no coincide con el esperado ({expected_profiles}).")
        # Verifica que cada clave y su pago asociado tengan la longitud correcta
        for key, valor in pagos.items():
            if len(key) != len(jugadores):
                raise ValueError(f"La clave {key} no coincide con el número de jugadores ({len(jugadores)}).")
            if len(valor) != len(jugadores):
                raise ValueError(f"Los pagos {valor} no coinciden con el número de jugadores ({len(jugadores)}).")
        return self

# ---------------------------
# Funciones auxiliares

# Retorna el pago para un perfil dado
def pagos_estrategia(juego: EstructuraDeJuego, perfil: Tuple[str, ...]) -> Tuple[float, ...]:
    if perfil not in juego.pagos:
        raise ValueError(f"El perfil {perfil} no está definido en los pagos del juego.")
    return juego.pagos[perfil]

# Extrae las matrices de pago para un juego de dos jugadores
def get_payoff_matrices(juego: EstructuraDeJuego):
    jugador1, jugador2 = juego.jugadores[0], juego.jugadores[1]
    estrategiasA = [e.nombre for e in juego.estrategias[jugador1]]
    estrategiasB = [e.nombre for e in juego.estrategias[jugador2]]
    m = len(estrategiasA)
    n = len(estrategiasB)
    payoffA = [[0 for _ in range(n)] for _ in range(m)]
    payoffB = [[0 for _ in range(n)] for _ in range(m)]
    for i, eA in enumerate(estrategiasA):
        for j, eB in enumerate(estrategiasB):
            key = (eA, eB)
            if key not in juego.pagos:
                raise ValueError(f"No hay pago definido para el perfil {key}.")
            p1, p2 = juego.pagos[key]
            payoffA[i][j] = p1
            payoffB[i][j] = p2
    return payoffA, payoffB, estrategiasA, estrategiasB

# Calcula los equilibrios de Nash en estrategias puras (para juegos de dos jugadores)
def findPureStrategyNashEquilibria(payoffA: List[List[float]], payoffB: List[List[float]]) -> List[Tuple[int, int]]:
    equilibria = []
    m = len(payoffA)
    n = len(payoffA[0]) if m > 0 else 0
    for i in range(m):
        for j in range(n):
            # Para Aitana (jugador 0)
            canAImprove = False
            for i_prime in range(m):
                if payoffA[i_prime][j] > payoffA[i][j]:
                    canAImprove = True
                    break
            # Para el Crítico (jugador 1)
            canBImprove = False
            for j_prime in range(n):
                if payoffB[i][j_prime] > payoffB[i][j]:
                    canBImprove = True
                    break
            if (not canAImprove) and (not canBImprove):
                equilibria.append((i, j))
    return equilibria

# Función envolvente para obtener los equilibrios de Nash
def nash_equilibria(juego: EstructuraDeJuego) -> List[Dict[str, Any]]:
    payoffA, payoffB, estrategiasA, estrategiasB = get_payoff_matrices(juego)
    indices_eq = findPureStrategyNashEquilibria(payoffA, payoffB)
    resultados = []
    for (i, j) in indices_eq:
        perfil = (estrategiasA[i], estrategiasB[j])
        pago = (payoffA[i][j], payoffB[i][j])
        resultados.append({
            "perfil": perfil,
            "pagos": pago
        })
    return resultados

# ---------------------------
# Función para detectar la estrategia dominante (en juegos de dos jugadores)

def dominant_strategy_for_player(juego: EstructuraDeJuego, player: str) -> str:
    i = juego.jugadores.index(player)
    # Extraer nombres de estrategias para el jugador
    strategies = [e.nombre for e in juego.estrategias[player]]
    # Identificar al oponente (en un juego de dos jugadores)
    opponent = juego.jugadores[1 - i]
    opponent_strategies = [e.nombre for e in juego.estrategias[opponent]]
    
    for s in strategies:
        es_dominante = True
        mejora_al_menos_una_vez = False
        for s_prime in strategies:
            if s_prime == s:
                continue
            # Para cada estrategia del oponente, comparar los pagos
            for t in opponent_strategies:
                # Construir el perfil según el orden
                if i == 0:
                    perfil_s = (s, t)
                    perfil_sprime = (s_prime, t)
                else:
                    perfil_s = (t, s)
                    perfil_sprime = (t, s_prime)
                payoff_s = juego.pagos[perfil_s][i]
                payoff_sprime = juego.pagos[perfil_sprime][i]
                if payoff_s < payoff_sprime:
                    es_dominante = False
                    break
                if payoff_s > payoff_sprime:
                    mejora_al_menos_una_vez = True
            if not es_dominante:
                break
        if es_dominante and mejora_al_menos_una_vez:
            return s
    return None

def dominant_strategy_equilibrium(juego: EstructuraDeJuego) -> List[Dict[str, Any]]:
    dominant = {}
    for player in juego.jugadores:
        ds = dominant_strategy_for_player(juego, player)
        if ds is None:
            # Si algún jugador no tiene estrategia dominante, se devuelve lista vacía.
            return []
        dominant[player] = ds
    # Construir el perfil dominante en el orden de juego.jugadores
    perfil = tuple(dominant[player] for player in juego.jugadores)
    pagos_obtenidos = juego.pagos[perfil]
    return [{
        "perfil": perfil,
        "pagos": pagos_obtenidos,
        "dominante": dominant
    }]

# ---------------------------
# Definición del juego: Aitana y Crítico

jugadores = ["Aitana", "Crítico"]

estrategias = {
    "Aitana": [Estrategia(nombre="Exponer"), Estrategia(nombre="No exponer")],
    "Crítico": [Estrategia(nombre="Critica positiva"), Estrategia(nombre="Critica negativa")]
}

# Los pagos se definen como:
# (Exponer, Critica positiva): (7.5, 7.5)
# (Exponer, Critica negativa): (9.8, 9.8)
# (No exponer, Critica positiva): (5.4, 5.4)
# (No exponer, Critica negativa): (8.2, 8.2)
pagos = {
    ("Exponer", "Critica positiva"): (7.5, 7.5),
    ("Exponer", "Critica negativa"): (9.8, 9.8),
    ("No exponer", "Critica positiva"): (5.4, 5.4),
    ("No exponer", "Critica negativa"): (8.2, 8.2)
}

juego_aitana = EstructuraDeJuego(
    nombre="Exposición de Aitana",
    descripcion=(
        "Aitana, una escultora que realizó una exposición sobre la neocolonización, "
        "tiene la opción de presentarse en el museo británico. Sin embargo, puede ser criticada: "
        "la crítica puede ser positiva o negativa. Los pagos para Aitana (y para el Crítico, "
        "asumiendo que sus preferencias sean similares) son los siguientes:\n"
        "- Si expone y recibe crítica positiva: 7.5\n"
        "- Si no expone y recibe crítica positiva: 5.4\n"
        "- Si expone y recibe crítica negativa: 9.8\n"
        "- Si no expone y recibe crítica negativa: 8.2"
    ),
    jugadores=jugadores,
    estrategias=estrategias,
    pagos=pagos
)

print("Juego definido correctamente:")
print(juego_aitana)

# ---------------------------
# Buscar y mostrar el equilibrio dominante (si existe)

eq_dominante = dominant_strategy_equilibrium(juego_aitana)
if eq_dominante:
    print("\nEquilibrio de Nash con estrategias dominantes encontrado:")
    for eq in eq_dominante:
        print(eq)
else:
    print("\nNo existe un equilibrio de Nash basado en estrategias dominantes en este juego.")

# En este juego, se espera que Aitana tenga como estrategia dominante 'Exponer'
# y el Crítico tenga como dominante 'Critica negativa', resultando en el perfil:
# ("Exponer", "Critica negativa") con pagos (9.8, 9.8).


Juego definido correctamente:
nombre='Exposición de Aitana' descripcion='Aitana, una escultora que realizó una exposición sobre la neocolonización, tiene la opción de presentarse en el museo británico. Sin embargo, puede ser criticada: la crítica puede ser positiva o negativa. Los pagos para Aitana (y para el Crítico, asumiendo que sus preferencias sean similares) son los siguientes:\n- Si expone y recibe crítica positiva: 7.5\n- Si no expone y recibe crítica positiva: 5.4\n- Si expone y recibe crítica negativa: 9.8\n- Si no expone y recibe crítica negativa: 8.2' jugadores=['Aitana', 'Crítico'] estrategias={'Aitana': [Estrategia(nombre='Exponer'), Estrategia(nombre='No exponer')], 'Crítico': [Estrategia(nombre='Critica positiva'), Estrategia(nombre='Critica negativa')]} pagos={('Exponer', 'Critica positiva'): (7.5, 7.5), ('Exponer', 'Critica negativa'): (9.8, 9.8), ('No exponer', 'Critica positiva'): (5.4, 5.4), ('No exponer', 'Critica negativa'): (8.2, 8.2)}

Equilibrio de Nash con es